In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")

In [109]:
#df = pd.read_csv('input/20190319_Board_ARC_Input_für_J.csv', sep=';', encoding='iso8859_15')
df = pd.read_csv('input/20190319_Board_ARC_Input_für_J_Datum_rckwrts.csv', sep=';', header=4, usecols=[0,1,4,5,6,7], parse_dates=[3,4,5], encoding='iso8859_15')

In [96]:
df.head()

,Art der Arbeit,Titel,Restaufwand [Tage],Next,In,Out
0,ARC-Fragestellungen entscheiden,Beschreibung des Prozess DL Beauftragung eConsult,10,NaT,2018-03-15,2018-07-27
1,ARC-Fragestellungen entscheiden,Beschreibung des Prozess DL Beauftragung Polyg...,10,NaT,2018-03-15,2018-07-27
2,Facharchitektur weiterenwickeln,Geschäftsfunktionen Übergabe an KOR,NaN,NaT,2018-06-15,2018-07-27
3,Lösungsarchitekturen entwickeln,Logging AAP,NaN,NaT,2018-05-01,2018-07-27
4,Produkt Auswahl begleiten,Kriterienkatalog Inpumanagement,NaN,NaT,2018-04-15,2018-07-27


In [99]:
df.dtypes

art              object
titel            object
plan             object
next     datetime64[ns]
input    datetime64[ns]
out      datetime64[ns]
dtype: object

In [110]:
df.columns = ['art', 'titel', 'plan', 'next', 'input', 'out']
df.art.drop_duplicates()

0                   ARC-Fragestellungen entscheiden
2                   Facharchitektur weiterenwickeln
3                   Lösungsarchitekturen entwickeln
4                         Produkt Auswahl begleiten
5                               Schulungen anbieten
8                     Lösungsarchitektur skizzieren
12                        ARC Governance entwickeln
16            In Communities (int & ext) einbringen
17              Iteraplan fachlich weiterentwickeln
19        Iterplan Software Pflege "Administration"
27                               Schulungsteilnahme
30                                        unbekannt
37                             Technologie bewerten
59     Zielarchitektur und IT-Strategien entwickeln
110                          ARC Review durchführen
122                                        Fastlane
126                                          intern
147                                  Regeltätigkeit
152                                          extern
Name: art, d

In [111]:
leer = np.isnat(df.input)
leer
#df[leer].input = df[leer].next

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
129    False
130    False
131    False
132    False
133    False
134    False
135    False
136    False
137    False
138    False
139    False
140    False
141    False
142    False
143    False
144    False
145    False
146    False
147    False
148    False
149    False
150    False
151    False
152    False
153    False
154    False
155    False
156    False
157    False
158    False
Name: input, Length: 159, dtype: bool

In [112]:
feiertage = ['2018-10-03', # Tag der Einheit
             '2018-11-01', # Allerheiligen
             '2018-12-24', # Heiligabend
             '2018-12-25', # Weihnachten
             '2018-12-26', # Weihnachten
             '2018-12-31', # Silvester
             '2019-01-01'  # Neujahr
            ]

# astype('datetime64[D]') notwendig
dauer=np.busday_count(df.input.values.astype('datetime64[D]'), 
                  df.out.values.astype('datetime64[D]'), 
                    holidays=feiertage)+1
dauer
#df[dauer>100]

,art,titel,plan,next,input,out
11,Lösungsarchitekturen entwickeln,Auswirkung AP der Zukunft auf P&C Claims und g...,NaN,NaT,2018-03-15,2018-08-15
42,unbekannt,Architektur Bipro,10,NaT,2018-01-01,2018-09-12
44,Lösungsarchitektur skizzieren,Projekt Mifid II Tapin Voranalyse,2,NaT,2018-02-15,2018-10-11
57,unbekannt,Steuerung TST Instanzaufbau,6,NaT,2018-03-01,2018-09-24
154,extern,61,3,NaT,2018-06-01,2019-01-29


In [113]:
df.tail()


,art,titel,plan,next,input,out
154,extern,61,3,NaT,2018-06-01,2019-01-29
155,extern,62,0,NaT,2019-01-28,2019-01-31
156,extern,63,0,2019-01-22,2019-01-24,2019-01-24
157,extern,64,0,NaT,2019-01-31,2019-02-06
158,extern,65,0,2019-01-31,2019-02-04,2019-02-14


In [114]:
df['dauer'] = dauer

In [115]:
von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

def leadtime(ldf, art, von, bis):
    #interpolation noch überprüfen
    perc80 = np.percentile(ldf.dauer,80,interpolation='lower')
    perc90 = np.percentile(ldf.dauer,90,interpolation='lower')
    
    # gca stands for 'get current axis'
    ax = plt.gca()
    #hist zählt direkt die Elemente
    ldf.plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, label='')
    #df[gewaehlt].plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, title='selected work item type')

    text90 = '90% fertig in ' + str(perc90) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc90+.1, linestyle='--', linewidth=1, color='red', label=text90)

    text80 = '80% fertig in ' + str(perc80) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc80+.1, linestyle='-.', linewidth=1, color='blue', label=text80)

    # Beschriftung
    plt.suptitle(art)
    #raus = str(von) +' bis ' + str(bis) + ': ' + str(len(ldf) + ' Zettel'
    title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
    plt.title(title)
    plt.xlabel('Dauer in Tagen')
    plt.ylabel('Anzahl')
    plt.legend(loc=0)
    
    #plt.show()
    filename='out/'+ art +'.png'
    plt.savefig(filename, dpi=300)
    plt.close()
#enddef plotte()

von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

for gew_art in df.art.drop_duplicates():
    gewaehlt = df.art == gew_art
#  nächste Zeile kann so nicht funktionieren
#    gewaehlt = df.input > '2018-12-31'
    leadtime(df[gewaehlt], gew_art, von, bis)
#endfor gew_art in ....

leadtime(df, 'Alle', von, bis)

In [1]:
arten[0]

NameError: name 'arten' is not defined